# **Connector**

In [ ]:
from google.colab import drive
import json, os
drive.mount('/content/drive')

Mounted at /content/drive


Conn to Data

In [ ]:
import json

path_in  = "/content/drive/MyDrive/Qwen LLM Share/Rubric/essay.json"   # แก้ path ให้ตรง
path_out = "/content/drive/MyDrive/Qwen LLM Share/Rubric/essay.jsonl"

# อ่านไฟล์ทั้งก้อน
with open(path_in, "r", encoding="utf-8") as f:
    s = f.read()

dec = json.JSONDecoder()
i = 0
n = len(s)

# หา '[' ตัวแรก (ต้องเป็น JSON array)
while i < n and s[i].isspace():
    i += 1
if i >= n or s[i] != "[":
    raise ValueError("ไฟล์นี้ไม่ใช่ JSON array (ไม่ได้ขึ้นต้นด้วย '[')")
i += 1  # ข้าม '['

wrote = 0
with open(path_out, "w", encoding="utf-8") as out:
    while True:
        # ข้ามช่องว่าง/ขึ้นบรรทัดใหม่
        while i < n and s[i].isspace():
            i += 1
        # ข้ามคอมมาคั่นรายการ (มีได้ทั้งหัวบรรทัด)
        if i < n and s[i] == ",":
            i += 1
            continue
        # จบรอบถ้าถึง ']'
        while i < n and s[i].isspace():
            i += 1
        if i < n and s[i] == "]":
            break
        if i >= n:
            break

        # พยายาม decode object ถัดไป
        try:
            obj, j = dec.raw_decode(s, i)
        except json.JSONDecodeError as e:
            # โชว์สไนเป็ตแถวที่พังไว้ไล่แก้ได้
            start = max(i - 120, 0)
            end   = min(i + 120, n)
            snippet = s[start:end].replace("\n", "\\n")
            raise RuntimeError(f"JSON พังแถว index {i}: {e}\n...{snippet}...")

        if isinstance(obj, dict):
            out.write(json.dumps(obj, ensure_ascii=False) + "\n")
            wrote += 1
        # ขยับ index ต่อไปหลังอ็อบเจ็กต์นี้
        i = j

print(f"เขียน JSONL เสร็จ: {path_out} (ทั้งหมด {wrote} แถว)")

เขียน JSONL เสร็จ: /content/drive/MyDrive/Qwen LLM Share/Rubric/essay.jsonl (ทั้งหมด 901 แถว)


QA Dataset

In [ ]:
import json

path = "/content/drive/MyDrive/Qwen LLM Share/Rubric/essay.jsonl"

# 1) นับจำนวนบรรทัด
with open(path, "r", encoding="utf-8") as f:
    lines = f.readlines()

print("จำนวนบรรทัดทั้งหมด:", len(lines))

# 2) เช็ค key ในแต่ละ record
bad = 0
for i, line in enumerate(lines, 1):
    try:
        obj = json.loads(line)
    except Exception as e:
        print(f"record {i} JSON พัง: {e}")
        bad += 1
        continue

    # เช็คว่ามี instruction, input, output
    for key in ["instruction", "input", "output"]:
        if key not in obj:
            print(f"record {i} ขาด key {key}")
            bad += 1

    # เช็คโครงสร้าง output
    if "output" in obj and isinstance(obj["output"], dict):
        for sub in ["strengths", "weaknesses", "scores", "summary"]:
            if sub not in obj["output"]:
                print(f"record {i} output ขาด {sub}")
                bad += 1

print("รวม error/warning:", bad)

# 3) print ตัวอย่าง 2 record แรก
for i in range(2):
    print(f"\n--- record {i+1} ---")
    print(lines[i][:500])

จำนวนบรรทัดทั้งหมด: 901
record 726 ขาด key instruction
รวม error/warning: 1

--- record 1 ---
{"instruction": "Give structured feedback for scholarship essay (JSON).", "input": "Growing up in a single-parent household, I watched my mother work three jobs to support our family. This ignited my passion for social work. Last year, I founded 'Bridge the Gap,' a nonprofit connecting low-income students with mentorship and scholarship resources. We've helped 47 students secure over $200,000 in aid. My goal is to earn my MSW and establish a national network of resource centers in underserved co

--- record 2 ---
{"instruction": "Give structured feedback for scholarship essay (JSON).", "input": "I want to be a doctor because I like helping people. I have always been interested in science and medicine. In high school, I was in the science club and volunteered at a hospital. I think doctors are important for society. My family doesn't have much money so I need this scholarship to pay for college

# **Hugging face**

loading

In [ ]:
from datasets import load_dataset

path = "/content/drive/MyDrive/Qwen LLM Share/Rubric/essay.jsonl"

ds = load_dataset("json", data_files=path, split="train")
print(ds)
print(ds[0])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', ',instruction'],
    num_rows: 901
})
{'instruction': 'Give structured feedback for scholarship essay (JSON).', 'input': "Growing up in a single-parent household, I watched my mother work three jobs to support our family. This ignited my passion for social work. Last year, I founded 'Bridge the Gap,' a nonprofit connecting low-income students with mentorship and scholarship resources. We've helped 47 students secure over $200,000 in aid. My goal is to earn my MSW and establish a national network of resource centers in underserved communities. This scholarship would cover my tuition gap of $8,000, allowing me to complete my degree debt-free and focus on expanding our organization's reach to five new states by 2027. Your foundation's commitment to educational equity perfectly aligns with my mission to break cycles of poverty through accessible education.", 'output': {'strengths': ['Strong personal narrative that connects experien

In [ ]:
# เอา dataset มาแล้ว drop column ที่ไม่ต้องการ
ds = ds.remove_columns([",instruction"])
print(ds)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 901
})


prepare for Qwen

In [ ]:
from datasets import load_dataset

path = "/content/drive/MyDrive/Qwen LLM Share/Rubric/essay.jsonl"
ds = load_dataset("json", data_files=path, split="train")

def format_example(ex):
    out = ex["output"]
    feedback = (
        "Strengths: " + "; ".join(out["strengths"]) + "\n"
        "Weaknesses: " + "; ".join(out["weaknesses"]) + "\n"
        "Scores: " + str(out["scores"]) + "\n"
        "Summary: " + out["summary"]
    )
    return {
        "prompt": f"Instruction: {ex['instruction']}\nEssay:\n{ex['input']}\n\nExpected Feedback:",
        "response": feedback
    }

ds = ds.map(format_example)
print(ds[0]["prompt"])
print(ds[0]["response"])

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

Instruction: Give structured feedback for scholarship essay (JSON).
Essay:
Growing up in a single-parent household, I watched my mother work three jobs to support our family. This ignited my passion for social work. Last year, I founded 'Bridge the Gap,' a nonprofit connecting low-income students with mentorship and scholarship resources. We've helped 47 students secure over $200,000 in aid. My goal is to earn my MSW and establish a national network of resource centers in underserved communities. This scholarship would cover my tuition gap of $8,000, allowing me to complete my degree debt-free and focus on expanding our organization's reach to five new states by 2027. Your foundation's commitment to educational equity perfectly aligns with my mission to break cycles of poverty through accessible education.

Expected Feedback:
Strengths: Strong personal narrative that connects experiences to career goals; Concrete evidence of leadership through founding a nonprofit with measurable impac

Qwen 1.8B

In [ ]:
!pip install trl peft accelerate bitsandbytes
!pip install -U transformers

In [ ]:
!pip install -U transformers trl peft accelerate bitsandbytes

In [ ]:
# === QLoRA + TRL SFT (trl==0.23.0) สำหรับ Qwen บน T4 ===
import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# os.environ["WANDB_MODE"] = "disabled"   # ไม่อยากให้ W&B โผล่ก็เปิดบรรทัดนี้

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

# ---- 0) เตรียม dataset (ต้องมีตัวแปร ds อยู่ก่อนแล้ว) ----
assert 'ds' in globals(), "ไม่พบตัวแปร ds — โหลด dataset ก่อนนะ"

def to_messages(ex):
    user = (
        "### Instruction\nGive structured feedback for scholarship essay (JSON).\n"
        "### Essay\n" + ex["input"] + "\n\n### Expected Feedback\n"
    )
    assistant = (
        "Strengths: " + "; ".join(ex["output"]["strengths"]) + "\n"
        "Weaknesses: " + "; ".join(ex["output"]["weaknesses"]) + "\n"
        "Scores: " + str(ex["output"]["scores"]) + "\n"
        "Summary: " + ex["output"]["summary"]
    )
    return {"messages": [
        {"role": "user", "content": user},
        {"role": "assistant", "content": assistant},
    ]}

chat_ds = ds.map(to_messages, remove_columns=ds.column_names)

# ---- 1) โมเดล + tokenizer ----
model_name = "Qwen/Qwen1.5-1.8B-Chat"   # ไม่ไหวค่อยลดเป็น 0.5B
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)
# กัน error pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.float16,
    quantization_config=bnb_config,
    trust_remote_code=True
)

# ---- 2) ใส่ LoRA ----
peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj","k_proj","o_proj"],  # เมมตึงค่อยเหลือ q/v
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# QLoRA trick + ประหยัด VRAM
model.enable_input_require_grads()
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
model.config.use_cache = False

# ---- 3) SFT config + Trainer ----
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(
    output_dir="./qwen-lora-essay",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=24,
    num_train_epochs=2,
    logging_steps=10,
    save_steps=200,
    save_total_limit=1,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,

    # ปิด AMP ให้พ้นๆ ไป
    fp16=False,
    bf16=False,
    optim="adamw_torch",

    max_length=256,
    packing=False,
    assistant_only_loss=False,
    report_to=[],
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=chat_ds,
    processing_class=tokenizer,
)

import torch; torch.cuda.empty_cache()
trainer.train()

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/901 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/901 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,4.467500
20,4.587800
30,4.538800
40,4.591500
50,4.523400
60,4.549500
70,4.523700


TrainOutput(global_step=76, training_loss=4.545303470210025, metrics={'train_runtime': 763.8588, 'train_samples_per_second': 2.359, 'train_steps_per_second': 0.099, 'total_flos': 4062913657208832.0, 'train_loss': 4.545303470210025, 'entropy': 1.9290468800336795, 'num_tokens': 443384.0, 'mean_token_accuracy': 0.3599717713388285, 'epoch': 2.0})

In [ ]:
# ที่เราใช้ SFT + PEFT อยู่ ตัว model คือ PEFT-wrapped แล้ว
out_dir = "/content/drive/MyDrive/Qwen LLM Share/Rubric/qwen-lora-essay"

trainer.save_model(out_dir)           # เซฟ LoRA adapter (ไม่ใช่ full base)
tokenizer.save_pretrained(out_dir)    # เผื่อโหลด inference
# ถ้ามี peft_config ตัวแปรนี้อยู่ด้วยก็เซฟไปด้วย (ถ้าไม่มี ข้ามได้)
try:
    from peft import LoraConfig
    peft_config.save_pretrained(out_dir)
except:
    pass

print("saved to:", out_dir)

saved to: /content/drive/MyDrive/Qwen LLM Share/Rubric/qwen-lora-essay


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPUs:", torch.cuda.device_count())

CUDA available: True
GPUs: 1


In [ ]:
import trl
print(trl.__version__)

0.23.0


Release

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/content/drive/MyDrive/Qwen LLM Share/Rubric/qwen-lora-essay",
    repo_id="NewJetsada/ScholarLens",   # ตรงนี้ใส่ชื่อ repo ที่มึงเพิ่งสร้าง
    repo_type="model"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 21.4kB / 6.32MB            

  ...lora-essay/tokenizer.json:   0%|          | 24.0kB / 11.4MB            

  ...a-essay/training_args.bin:   9%|9         |   574B / 6.16kB            

CommitInfo(commit_url='https://huggingface.co/NewJetsada/ScholarLens/commit/d22fa7d3a58c67d34aa282e1e91357b2de5843c3', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d22fa7d3a58c67d34aa282e1e91357b2de5843c3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NewJetsada/ScholarLens', endpoint='https://huggingface.co', repo_type='model', repo_id='NewJetsada/ScholarLens'), pr_revision=None, pr_num=None)

In [ ]:
trainer.save_model("/content/drive/MyDrive/Qwen LLM Share/Rubric/qwen-lora-essay")

In [ ]:
repo_id = "NewJetsada/ScholarLens"
out_dir = "/content/drive/MyDrive/Qwen LLM Share/Rubric/qwen-lora-essay"

readme = f"""---
license: apache-2.0
tags:
- qwen
- lora
- qlora
- scholarship-essay
base_model: Qwen/Qwen1.5-1.8B-Chat
datasets:
- custom/scholarship-essay-feedback
---

# {repo_id}

LoRA adapter สอน Qwen ให้ให้ feedback แบบ JSON สำหรับ **scholarship essays**.
ฝึกด้วย ~901 ตัวอย่าง (synthetic + curated).

## ใช้ยังไง (โหลด adapter)
```python
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

base = "Qwen/Qwen1.5-1.8B-Chat"
adapter = "{repo_id}"

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)

bnb = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(base, quantization_config=bnb, device_map="auto", trust_remote_code=True)
model = PeftModel.from_pretrained(model, adapter)
model.eval()"""

In [ ]:
with open(f"{out_dir}/README.md", "w", encoding="utf-8") as f:
    f.write(readme)

from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path=out_dir,
    repo_id=repo_id,
    repo_type="model"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...a-essay/training_args.bin: 100%|##########| 6.16kB / 6.16kB            

  ...adapter_model.safetensors: 100%|##########| 6.32MB / 6.32MB            

  ...lora-essay/tokenizer.json:  73%|#######2  | 8.29MB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/NewJetsada/ScholarLens/commit/6f18c3c0ba129ffaf360961a73571d27781106e2', commit_message='Upload folder using huggingface_hub', commit_description='', oid='6f18c3c0ba129ffaf360961a73571d27781106e2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NewJetsada/ScholarLens', endpoint='https://huggingface.co', repo_type='model', repo_id='NewJetsada/ScholarLens'), pr_revision=None, pr_num=None)

In [ ]:
trainer.save_model("/content/drive/MyDrive/Qwen LLM Share/Rubric/qwen-lora-essay")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

base = "Qwen/Qwen1.5-1.8B-Chat"
adapter = "NewJetsada/ScholarLens"   # repo ที่มึงสร้าง

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    base, quantization_config=bnb, device_map="auto", trust_remote_code=True
)
model = PeftModel.from_pretrained(model, adapter)
model.eval()

adapter_config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.32M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [ ]:
essay = """Growing up in a single-parent household, I watched my mother work three jobs...
This ignited my passion for social work...
"""

prompt = f"""### Instruction
Give structured feedback for scholarship essay (JSON).
### Essay
{essay}

### Expected Feedback
"""

In [ ]:
inputs = tok(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=300, temperature=0.7)

print(tok.decode(out[0], skip_special_tokens=True))

### Instruction
Give structured feedback for scholarship essay (JSON).
### Essay
Growing up in a single-parent household, I watched my mother work three jobs...
This ignited my passion for social work...


### Expected Feedback
```json
{
  "title": "Structured Feedback for Scholarship Essay",
  "description": [
    {
      "paragraph": "Title: Growing Up in a Single-Parent Household and the Inspiration for Social Work",
      "feedback": "The essay effectively describes your personal experience growing up in a single-parent household. You have captured the essence of this situation by mentioning the challenges you faced, such as having to work multiple jobs to support your family. This anecdote serves as a significant source of inspiration for pursuing a career in social work. By highlighting your mother's determination to provide for her family despite the hardships she faced, the essay demonstrates your strong sense of empathy and compassion."
    },
    {
      "paragraph": "Paragra

In [ ]:
   essay = """Growing up in a single-parent household, I watched my mother work three jobs...
This ignited my passion for social work..."""

prompt = f"""### Instruction
Give structured feedback for scholarship essay (JSON).
### Essay
{essay}

### Expected Feedback
"""

inputs = tok(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id or tok.eos_token_id,
    )

raw = tok.decode(out[0], skip_special_tokens=True)

# ===== ดึง JSON หลัง assistant =====
def extract_json_block(s: str) -> str:
    # ตัดพวก system/user ออก
    if "assistant" in s:
        s = s.split("assistant", 1)[-1]
    # ล้าง code fence
    s = re.sub(r"```(?:json)?", "", s)
    s = re.sub(r"```", "", s)
    # หา block { ... }
    start, depth, cand = -1, 0, None
    for i, ch in enumerate(s):
        if ch == "{":
            if depth == 0: start = i
            depth += 1
        elif ch == "}":
            depth -= 1
            if depth == 0 and start != -1:
                cand = s[start:i+1]
                break
    return cand

json_str = extract_json_block(raw)
data = json.loads(json_str) if json_str else None

print("---- RAW ----")
print(raw)
print("\n---- JSON ----")
print(json.dumps(data, ensure_ascii=False, indent=2))

---- RAW ----
### Instruction
Give structured feedback for scholarship essay (JSON).
### Essay
Growing up in a single-parent household, I watched my mother work three jobs...
This ignited my passion for social work...

### Expected Feedback
1. Start with a clear introduction that hooks the reader's attention and explains why you are applying for the scholarship.
```json
{
  "title": "Application for Scholarship",
  "description": "Growing up in a single-parent household, I watched my mother work three jobs to provide for our family. This experience taught me the importance of resilience and determination in achieving success.",
  "requirements": [
    {
      "field": "Introduction",
      "description": "Start with a clear introduction that hooks the reader's attention and explains why you are applying for the scholarship. Include a brief overview of your background, including your single-parent upbringing and your interest in social work."
    },
    {
      "field": "Personal Story"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from transformers import pipeline
import torch, json

base = "Qwen/Qwen1.5-1.8B-Chat"
adapter = "NewJetsada/ScholarLens"

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)
bnb = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(
    base, quantization_config=bnb, device_map="auto", trust_remote_code=True
)
model = PeftModel.from_pretrained(model, adapter)
model.eval()

essay = """Growing up in a single-parent household, I watched my mother work three jobs...
This ignited my passion for social work..."""

# === ใช้ HF pipeline + enforce JSON ===
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    device_map="auto"
)

messages = [
    {"role": "system", "content": "Return ONLY a valid JSON object with this schema:\n"
     "{ 'strengths': [string], 'weaknesses': [string], "
     "'scores': {'structure': int, 'specificity': int, 'evidence': int, 'clarity': int, 'tone': int}, "
     "'summary': string }"
     "\nRules: Integers 1..5 only. No prose. No code fences. JSON only."},
    {"role": "user", "content": f"Essay:\n{essay}"}
]

out = pipe(messages, max_new_tokens=300, do_sample=False)[0]["generated_text"]

# ตรงนี้ HuggingFace pipeline จะคืนมาเป็น dict อยู่แล้ว (chat template)
# แต่ถ้าเป็น string ยาวๆ ก็พยายาม parse JSON ตรงๆ
try:
    data = json.loads(out[-1]["content"])  # อันสุดท้ายคือ assistant
    print("✅ Parsed JSON:")
    print(json.dumps(data, ensure_ascii=False, indent=2))
except Exception as e:
    print("❌ Fail parse:", e)
    print("RAW:", out)

Device set to use cuda:0


✅ Parsed JSON:
{
  "strengths": [
    "Hardworking",
    "Resourceful",
    "Empathetic"
  ],
  "weaknesses": [
    "Lack of financial stability",
    "Multiple responsibilities",
    "Time management challenges"
  ],
  "scores": {
    "structure": 4,
    "specificity": 3,
    "evidence": 2,
    "clarity": 3,
    "tone": 4
  },
  "summary": "Growing up in a single-parent household, my mother worked multiple jobs to provide for our family. This instilled in me a strong work ethic and resourcefulness, as well as an empathetic nature towards others who may be facing similar struggles. Despite the challenges she faced, such as financial instability and time management, she was able to excel in her profession and make a positive impact on the lives of those around her."
}
